In [1]:
import requests
from pymongo import MongoClient
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib
import sys

%load_ext autoreload
%autoreload 2

## To write down all the urls to be scraped for articles and comments

In [4]:
# url = "https://www.theguardian.com/us/commentisfree"

# Which links to look for.


get_urls()

NameError: name 'get_urls' is not defined

## To download comments from guardian website

In [10]:
def download_comments(guardianLink):
    '''
    This function downloads the comments, along with title, topic of the original thread
    comment_id, no_upvotes, no_replies, comment_author, comment_author_id, etc
    Returns BeautifulSoup; need more extraction from the soup for the above-mentioned 
    parameters
    
    '''
        
    #Figuring out the PageId
    pageResponse = urllib2.urlopen(guardianLink)
    commentsMatch = re.search(r'/p/(.*?)"', pageResponse.read(), re.M|re.I)
    # print "Comments Match = ", commentsMatch()
    if commentsMatch:
        pageId = commentsMatch.group(1)
        print '[+] pageId has been retrieved ('+pageId+')'
    else:
        sys.exit('[-] Could not retrieve pageId!')

    #Retrieving comments
    downloadCount = 1
    downloadError = 0

    text = ''

    while downloadError == 0:
        try:
            response = urllib2.urlopen('http://www.theguardian.com/discussion/p/' + pageId 
                                       + '?page=' + str(downloadCount))
            html = response.read()
            length = len(text)
            text += html
            downloadCount = downloadCount + 1

        except:
            downloadError = 1

    return BeautifulSoup(text, 'html.parser')

## To extract data from soup

#### Returns a list of lists - comment_od, comment_text, author_id, author_name, number_of_upvotes

In [11]:
def getting_comment_data(soup):
    '''
    This function takes in the soup of comments. Returns a list of list. 
    Each element in the list is a list of -
    comment_id, comment, author_id, author, number_of_upvotes
    
    '''
       
    auth_name_lst = []
    comm_id_lst = []
    auth_id_lst = []
    for lis in soup.find_all('li'):
        if 'data-comment-author-id' in (lis.attrs) and 'data-comment-id' in (lis.attrs)and 'data-comment-author' in (lis.attrs):
            auth_name_lst.append(lis.attrs['data-comment-author'].encode('utf-8').replace("  ", " "))
            auth_id_lst.append(int(lis.attrs['data-comment-author-id']))
            comm_id_lst.append(int(lis.attrs['data-comment-id']))

    comments_text = soup.findAll("div", { "class" : "d-comment__body" })
    recommends = soup.findAll("span", {"class" : "d-comment__recommend-count--old"})
    users = soup.findAll("span", {"itemprop" : "givenName"})

    comment_data_list = []
    comment_text_list = []
    comment_id_list = []
    author_id_list = []
    author_name_list = []
    upvotes_count_list = []

    i = 0
    j = 0
    for comment_text, upvotes, user, auth_name, auth_id, comment_id in zip(comments_text, recommends, users, auth_name_lst, auth_id_lst, comm_id_lst):
        i += 1
        if 'comment was removed by a moderator ' not in comment_text.text:
            j += 1
#             This is the count of comments not removed by the moderator
                    

#             if auth_name.strip() != user.text.encode('utf-8').strip():
#                 print "something is broken for -"+ auth_name.strip()+ "-"+ user.text.encode('utf-8').strip()
#             else :
            if not upvotes.txt:
                upvote = 0
            else : 
                upvote = int(upvotes.txt)
            comment_text_list.append(comment_text.text)
            comment_id_list.append(comment_id)
            author_id_list.append(auth_id)
            author_name_list.append(auth_name)
            upvotes_count_list.append(upvote)            

    return comment_text_list, comment_id_list, author_id_list, auth_name_lst, upvotes_count_list

In [12]:
url = "https://www.theguardian.com/commentisfree/2017/aug/13/america-carpet-bombed-north-korea-remember-that-past"
soup = download_comments(url)
getting_comment_data(soup)

[+] pageId has been retrieved (72e9y)


([u'\nSay what?And do you know how someone who wants something but is not willing to do the hard work is called anywhere in the world (outside maybe Trumpland)?\n',
  u'\nGood. I will be sure to repeat that sentence hourly from no one, lest I be mistaken for a North Korean double agent again...\n',
  u'\nYes.\n',
  u'\nAnd your original claim that a large percentage of comments here suggest that people don\'t think North Korea is "bad bad ba" is utterly laughable.\n',
  u"\n\nNorth Korea is very bad and South Korea is not.\n\nDon't know what you mean. Literally no idea. Nothing on this planet has ever been that simple. You sound like Donald Trump.\nNorth Korea is a terrifying, totalitarian hell-hole, led by a brutal family dynasty that maintains the status-quo purely for their own material gains, and will happily keep its populace in fear and poverty as long as they keep power, even if that means chucking nuclear weapons at innocent civilians. Happy now?\n",
  u'\nDallasWilliams: " ...

### To extract topic(s) and text of an article

In [13]:
def article_topics_title(url):
    '''
    Given a URL, this function returns topics and contents of the article
    
    '''

    soup = BeautifulSoup(requests.get(url).text, 'html.parser')

    topic_list = [topic.attrs['data-link-name'][9:] for topic in soup.findAll("a", {"class" : "submeta__link"})
        if 'data-link-name' in topic.attrs and 'keyword: ' in topic.attrs['data-link-name']]

    for title in soup.find("h1", {"class" : "content__headline"}):
        title = str(title.strip().encode('utf-8'))
        

    for tag in soup.find_all('svg'):
        tag.decompose()
    for tag in soup.find_all('figure'):
        tag.decompose()
    for tag in soup.find_all('aside'):
        tag.decompose()
    for tag in soup.find_all('span'):
        tag.decompose()
    
    article = [str(s) for s in soup.find("div", {"class" : "content__article-body from-content-api js-article__body"})]
    
  
    return topic_list, article, title

In [16]:
urls.count()

2

In [14]:
def getting_all_data():
    cursor = urls.find({})
    for document in cursor: 
        url = str(document['url'])
        id_n = document['_id']
        print "Working on this link", url

        soup = download_comments(url)
        #     comment_text_list, comment_id_list, author_id_list, auth_name_lst, upvotes_count_list
        comment_text_list, comment_id_list, author_id_list, auth_name_lst, upvotes_count_list = getting_comment_data(soup)

        topics_list, article, title = article_topics_title(url)
        print "Title of the article is: ", title

        urls.update_one({'url': url}, 
                        {"$set": {'title': title, 'article' : article, 'topics_list': topics_list,
                                 'comment_ids' : comment_id_list, 'comment_text' : comment_text_list,
                                 'author_ids' : author_id_list, 'author_name' : auth_name_lst,
                                 'upvotes' : upvotes_count_list}}, upsert=True)
        return
    
getting_all_data()

Working on this link https://www.theguardian.com/commentisfree/2017/jan/31/america-theresa-may-condemn-trump-travel-ban-controlling-borders
[+] pageId has been retrieved (5pxhq)
Title of the article is:  As a friend of America, Theresa May must condemn Trump’s travel ban


In [52]:
urls.count()

/home/sanhita/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


38

In [6]:
from pymongo import MongoClient

client = MongoClient()

db = client["guardian"] #This is the name of the database
urls = db["urls"] # this is the table in that database
data = db["data"]
from pprint import pprint
cursor = urls.find({})
# i = 0
# for document in cursor: 
#     if i < 20:
#         for k, v in document.iteritems():
#             print k, v
#     i += 1
#     pprint(document['upvotes'])


In [22]:
cursor = urls.find({})
count_url = 0
count_title = 0
for document in cursor: 
    if len(document.keys()) == 3:
        print (document['_id'], document['id'])
        count_url += 1
        if count_url > 3:
            break
    
#       url = str(document['url'])
        
    
#     count_title += len(document['_id'])


print (document)
# print (document['_id'], document['id'], document['url'])

5c2fc2a3e9e22416aaea25ca 0
5c2fc2a3e9e22416aaea25cb 1
5c2fc2a3e9e22416aaea25cc 2
5c2fc2a3e9e22416aaea25cd 3
{'_id': ObjectId('5c2fc2a3e9e22416aaea25cd'), 'id': 3, 'url': 'https://www.theguardian.com/commentisfree/2018/jul/13/killing-blue-whale-disconnected-nature-sea-ignorance'}


In [6]:
def get_urls():
    years = ["2018"]
    months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
    dates = [str(n) for n in range(1,28)]


    years = ["2018"]
    months = ["dec"]
    dates = [str(31)]


    # count of number of links
    i = 0

    # Initiating mongoDB. Using pymongo to connect the database
    client = MongoClient()
    db = client["guardian"] #This is the name of the database
    urls = db["urls"] # this is the table in that database

    result = db.urls.delete_many({}) # A fresh start to the DB table -> removing all entries
    all_links = []

    for year in years:
        for month in months:
            for date in dates:
                root_url = "https://www.theguardian.com/commentisfree/" + year  + "/" + month + "/" + date

                # This one contains all the URLs. Soup extracts those.
                list_of_urls = requests.get(root_url).text
                soup = BeautifulSoup(list_of_urls, "html.parser")

                # the .findAll method from re finds the _content_ that has
                # html tag - 'a', attribute - 'href' and
                # the pattern that begins with the root_url variable
                for link in soup.findAll('a', attrs={'href': re.compile(root_url)}):
                    all_links.append(link.get('href'))

    all_links = set(all_links)
    for n, link in enumerate(all_links):
        # urls is the name of the mongo DataBase. insert_one is the method
        #if n < 2:
        urls.insert_one({"id" : i, "url" : link})
    #             print link
        i += 1
    print ("Total number of links stored is ", i)

    return

In [7]:
get_urls()

Total number of links stored is  16


In [3]:
from writing_database import article_topics_title, real_download

In [18]:
real_download()

Title of the article is:  b'Don\xe2\x80\x99t expect Brexit to give us a British Alexander Hamilton'


Title of the article is:  b'The uncontacted tribes of Brazil face genocide under Jair Bolsonaro'


Message : 2 documents downloaded.
Title of the article is:  b'There is a path to a second referendum \xe2\x80\x93 and only Labour can win it'




KeyboardInterrupt: 

In [62]:
urls.find_one()

{'_id': ObjectId('5c2fe9cbe9e224184f904f16'),
 'id': 0,
 'url': 'https://www.theguardian.com/commentisfree/2018/dec/31/happy-new-year-natural-environmental-damage'}

In [63]:
topic_list, article, title = article_topics_title(urls.find_one()['url'])

In [64]:
topic_list

['environment/whaling',
 'world/natural-disasters',
 'environment/conservation',
 'world/animals',
 'environment/marine-life',
 'environment/whales']

In [66]:
title

"b'Happy new year? Tell that to the natural world we are destroying'"

In [71]:
urls.find_one()

{'_id': ObjectId('5c2fe9cbe9e224184f904f16'),
 'id': 0,
 'url': 'https://www.theguardian.com/commentisfree/2018/dec/31/happy-new-year-natural-environmental-damage'}

In [85]:
res = data.insert_one({'url' : url, 'article' : article, 
                       'title' : title, 'topic_list' : topic_list})

In [86]:
res.inserted_id

ObjectId('5c2fef56e9e224184f904f41')

In [88]:
res.acknowledged

True

In [20]:
for d in data.find():
    print (d['_id'])

5c2ffa84e9e224288fc9176f
5c2ffaabe9e224288fc91770
5c2ffac3e9e224288fc91771
5c2ffae7e9e224288fc91772
5c2ffaf5e9e224288fc91773
5c2ffb1ce9e224288fc91774
5c2ffb38e9e224288fc91775
5c2ffb49e9e224288fc91776
5c2ffb54e9e224288fc91777
5c2ffb62e9e224288fc91778
5c2ffb7fe9e224288fc91779
5c2ffba4e9e224288fc9177a
5c2ffbb6e9e224288fc9177b
5c2ffbc1e9e224288fc9177c
5c2ffbd0e9e224288fc9177d
5c2ffbd9e9e224288fc9177e
5c2ffbe6e9e224288fc9177f
5c2ffbf6e9e224288fc91780
5c2ffc0ce9e224288fc91781
5c2ffc1ee9e224288fc91782
5c2ffc25e9e224288fc91783
5c2ffc2fe9e224288fc91784
5c2ffc48e9e224288fc91785
5c2ffc53e9e224288fc91786
5c2ffc5de9e224288fc91787
5c2ffc72e9e224288fc91788
5c2ffc97e9e224288fc91789
5c2ffcb8e9e224288fc9178a
5c2ffccbe9e224288fc9178b


In [22]:
data.find_one({'id' : 3})

{'_id': ObjectId('5c2ffae7e9e224288fc91772'),
 'article': ['\n',
  '<p>n 1982, when I was having a bit of an existential crisis while campaigning for nuclear disarmament and heavily pregnant, my father-in-law related the story of a neighbour who’d shot dead his wife, children and himself during the Cuba crisis because he couldn’t face the prospect of inevitable nuclear war. “He could have been enjoying his grandchildren now.”</p>',
  '\n',
  '<p>Climate change (<a class="u-underline" data-link-name="in body link" href="https://www.theguardian.com/commentisfree/2018/aug/19/its-the-end-of-the-earth-as-we-know-it-read-all-about-it" title="">It’s the end of the Earth as we know it</a>, Comment) appears to momentous, and not withstanding the depressing (and logical) thoughts that it’s too late, too complex, too difficult and so forth, I still reckon it’s a good strategy to act as if changes in human behaviour, individual and collective, on any scale – massive and radical would be good optio